In [48]:
from datadreamer import DataDreamer
from datadreamer.llms import OpenAI
from datadreamer.steps import DataFromPrompt, Embed, CosineSimilarity, concat, HFHubDataSource
from datadreamer.embedders import SentenceTransformersEmbedder
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import matplotlib.pyplot as plt
import math
import re
import pandas as pd
# from scipy.spatial.distance import cosine

In [22]:
with DataDreamer("./output"):
    stel_dataset = HFHubDataSource(
        "Lexical Features",
        path="jjz5463/probing_dataset_4.0",
        split="train"
    )

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'Lexical Features' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: ./output


In [46]:
def compute_embeddings(
        dataset_pos, dataset_neg, model: str
):
    with DataDreamer("./output"):
        pos_embedded_data = Embed(
            name = f"{model.replace('/', ' ')} Embeddings for Positive Examples",
            inputs = {
                "texts": dataset_pos
            },
            args = {
                "embedder": SentenceTransformersEmbedder(
                    model_name=model
                ),
                "truncate": True
            },
            outputs = {
                "texts": "sentences",
                "embeddings": "embeddings"
            },
        )
        neg_embedded_data = Embed(
            name = f"{model.replace('/', ' ')} Embeddings for Negative Examples",
            inputs = {
                "texts": dataset_neg
            },
            args = {
                "embedder": SentenceTransformersEmbedder(
                    model_name=model
                ),
                "truncate": True
            },
            outputs = {
                "texts": "sentences",
                "embeddings": "embeddings"
            },
        )
    return pos_embedded_data, neg_embedded_data

def convert_embeddings(pos_embedded_data, neg_embedded_data):
    pos_formal_embeddings = np.array(pos_embedded_data.output["embeddings"][0:100])
    neg_formal_embeddings = np.array(neg_embedded_data.output["embeddings"][0:100])
    paired_formal_embeddings = [(pos, neg) for pos, neg in zip(pos_formal_embeddings, neg_formal_embeddings)]
    pos_complex_embeddings = np.array(pos_embedded_data.output["embeddings"][100:200])
    neg_complex_embeddings = np.array(neg_embedded_data.output["embeddings"][100:200])
    paired_complex_embeddings = [(pos, neg) for pos, neg in zip(pos_complex_embeddings, neg_complex_embeddings)]
    pos_contraction_embeddings = np.array(pos_embedded_data.output["embeddings"][200:300])
    neg_contraction_embeddings = np.array(neg_embedded_data.output["embeddings"][200:300])
    paired_contraction_embeddings = [(pos, neg) for pos, neg in zip(pos_contraction_embeddings, neg_contraction_embeddings)]
    pos_number_embeddings = np.array(pos_embedded_data.output["embeddings"][300:400])
    neg_number_embeddings = np.array(neg_embedded_data.output["embeddings"][300:400])
    paired_number_embeddings = [(pos, neg) for pos, neg in zip(pos_number_embeddings, neg_number_embeddings)]
    return paired_formal_embeddings, paired_complex_embeddings, paired_contraction_embeddings, paired_number_embeddings

def compute_accuracy(paired_embeddings: list):
    accuracy = 0
    correct = 0
    rand = 0
    incorrect = 0
    for i in range(len(paired_embeddings)):
        anchor_pos, anchor_neg = paired_embeddings[i]
        norm_anchor_pos, norm_anchor_neg = anchor_pos / np.linalg.norm(anchor_pos), anchor_neg / np.linalg.norm(anchor_neg)
        # anchor_pos = anchor_pos.reshape(1, -1)
        # anchor_neg = anchor_neg.reshape(1, -1)
        for j in range(i+1, len(paired_embeddings)):
            alt_pos, alt_neg = paired_embeddings[j]
            norm_alt_pos, norm_alt_neg = alt_pos / np.linalg.norm(alt_pos), alt_neg / np.linalg.norm(alt_neg)
            # alt_pos = alt_pos.reshape(1, -1)
            # alt_neg = alt_neg.reshape(1, -1)
            
            sim1 = np.dot(norm_anchor_pos, norm_alt_pos)
            sim2 = np.dot(norm_anchor_neg, norm_alt_neg)
            sim3 = np.dot(norm_anchor_pos, norm_alt_neg)
            sim4 = np.dot(norm_anchor_neg, norm_alt_pos)
            # sim1 = cosine_similarity(anchor_pos, alt_pos)[0][0]
            # sim2 = cosine_similarity(anchor_neg, alt_neg)[0][0]
            # sim3 = cosine_similarity(anchor_pos, alt_neg)[0][0]
            # sim4 = cosine_similarity(anchor_neg, alt_pos)[0][0]
            if math.pow(1 - sim1, 2) + math.pow(1 - sim2, 2) == math.pow(1 - sim3, 2) + math.pow(1 - sim4, 2):
                accuracy += 0.5
                rand += 1
            elif math.pow(1 - sim1, 2) + math.pow(1 - sim2, 2) < math.pow(1 - sim3, 2) + math.pow(1 - sim4, 2):
                accuracy += 1
                correct += 1
            else:
                accuracy += 0
                incorrect += 1
    return accuracy / (len(paired_embeddings) * (len(paired_embeddings) - 1) / 2)

def STEL_benchmark(dataset_pos, dataset_neg, model):
    pos_embedded_data, neg_embedded_data = compute_embeddings(dataset_pos, dataset_neg, model)
    paired_formal_embeddings, paired_complex_embeddings, paired_contraction_embeddings, paired_number_embeddings = convert_embeddings(pos_embedded_data, neg_embedded_data)
    formal_accuracy = compute_accuracy(paired_formal_embeddings)
    complex_accuracy = compute_accuracy(paired_complex_embeddings)
    contraction_accuracy = compute_accuracy(paired_contraction_embeddings)
    number_accuracy = compute_accuracy(paired_number_embeddings)
    avg_accuracy = (formal_accuracy + complex_accuracy + contraction_accuracy + number_accuracy) / 4
    return formal_accuracy, complex_accuracy, contraction_accuracy, number_accuracy, avg_accuracy

def STEL_print(model):
    formal_accuracy, complex_accuracy, contraction_accuracy, number_accuracy, avg_accuracy = STEL_benchmark(stel_dataset.output["positive"], stel_dataset.output["negative"], model)
    print(f"Formal Accuracy for {model} Embeddings: {formal_accuracy}")
    print(f"Complex Accuracy for {model} Embeddings: {complex_accuracy}")
    print(f"Contraction Accuracy for {model} Embeddings: {contraction_accuracy}")
    print(f"Number Accuracy for {model} Embeddings: {number_accuracy}")
    print(f"Average Accuracy for {model} Embeddings: {avg_accuracy}")

def STEL_table(model):
    formal_accuracy, complex_accuracy, contraction_accuracy, number_accuracy, avg_accuracy = STEL_benchmark(stel_dataset.output["positive"], stel_dataset.output["negative"], model)
    data = {
        'Metric': ['Formal Accuracy', 'Complex Accuracy', 'Contraction Accuracy', 'Number Accuracy', 'Average Accuracy'],
        f'{model} Embeddings': [formal_accuracy, complex_accuracy, contraction_accuracy, number_accuracy, avg_accuracy]
    }
    df = pd.DataFrame(data)
    print(df)


In [49]:
STEL_table("AnnaWegmann/Style-Embedding")
STEL_table("sentence-transformers/all-mpnet-base-v2")
STEL_table("google-bert/bert-base-uncased")
STEL_table("google-bert/bert-base-cased")
STEL_table("google-bert/bert-base-multilingual-cased")

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'AnnaWegmann Style-Embedding Embeddings for Positive Examples' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Step 'AnnaWegmann Style-Embedding Embeddings for Negative Examples' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: ./output
[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'sentence-transformers all-mpnet-base-v2 Embeddings for Positive Examples' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Step 'sentence-transformers all-mpnet-base-v2 Embeddings for Negative Examples' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: ./output


                 Metric  AnnaWegmann/Style-Embedding Embeddings
0       Formal Accuracy                                0.910505
1      Complex Accuracy                                0.601717
2  Contraction Accuracy                                0.946263
3       Number Accuracy                                0.908687
4      Average Accuracy                                0.841793


[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'google-bert bert-base-uncased Embeddings for Positive Examples' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Step 'google-bert bert-base-uncased Embeddings for Negative Examples' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: ./output


                 Metric  sentence-transformers/all-mpnet-base-v2 Embeddings
0       Formal Accuracy                                           0.881010 
1      Complex Accuracy                                           0.564747 
2  Contraction Accuracy                                           0.683838 
3       Number Accuracy                                           0.767475 
4      Average Accuracy                                           0.724268 


[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'google-bert bert-base-cased Embeddings for Positive Examples' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Step 'google-bert bert-base-cased Embeddings for Negative Examples' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: ./output


                 Metric  google-bert/bert-base-uncased Embeddings
0       Formal Accuracy                                  0.983838
1      Complex Accuracy                                  0.702525
2  Contraction Accuracy                                  0.909899
3       Number Accuracy                                  0.985051
4      Average Accuracy                                  0.895328


[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'google-bert bert-base-multilingual-cased Embeddings for Positive Examples' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Step 'google-bert bert-base-multilingual-cased Embeddings for Negative Examples' results loaded from disk. 🙌 It was previously run and saved.
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: ./output


                 Metric  google-bert/bert-base-cased Embeddings
0       Formal Accuracy                                0.974545
1      Complex Accuracy                                0.697071
2  Contraction Accuracy                                0.943838
3       Number Accuracy                                0.983232
4      Average Accuracy                                0.899672
                 Metric  google-bert/bert-base-multilingual-cased Embeddings
0       Formal Accuracy                                           0.988687  
1      Complex Accuracy                                           0.642323  
2  Contraction Accuracy                                           0.944646  
3       Number Accuracy                                           0.940404  
4      Average Accuracy                                           0.879015  
